In [1]:
import torch
import torchvision
import syft as sy
import pandas as pd
from torch import nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader, TensorDataset

In [2]:
dataframe_raw = pd.read_csv('MNIST/mnist_train.csv')
inputs = dataframe_raw.drop('label',axis=1).values/255
targets = dataframe_raw[['label']].values
inputs = inputs.reshape(-1,1,28,28)
dataset_train = TensorDataset(torch.tensor(inputs,dtype=torch.float32),torch.tensor(targets,dtype=torch.float32))

In [3]:
dataframe_raw = pd.read_csv('MNIST/mnist_test.csv')
inputs = dataframe_raw.drop('label',axis=1).values/255
targets = dataframe_raw[['label']].values
inputs = inputs.reshape(-1,1,28,28)
dataset_test = TensorDataset(torch.tensor(inputs,dtype=torch.float32),torch.tensor(targets,dtype=torch.float32))

In [4]:
hook = sy.TorchHook(torch)

In [5]:
jake = sy.VirtualWorker(hook, id="jake")
john = sy.VirtualWorker(hook, id="john")

In [6]:
federated_train_loader = sy.FederatedDataLoader(
    dataset_train.federate((jake, john)), batch_size=64, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    dataset_test, batch_size=64, shuffle=True)

In [7]:
def show_batch(dl):
    for images, labels in dl:
        print(images.size())
        print(labels.size())
        break

In [8]:
show_batch(test_loader)

torch.Size([64, 1, 28, 28])
torch.Size([64, 1])


In [9]:
# define the args
args = {
    'use_cuda' : True,
    'batch_size' : 64,
    'test_batch_size' : 1000,
    'lr' : 0.01,
    'log_interval' : 10,
    'epochs' : 10
}
use_cuda = args['use_cuda'] and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [10]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels = 1, out_channels = 32, kernel_size = 3, stride = 1),
            nn.ReLU(),
            nn.Conv2d(in_channels=32,out_channels = 64, kernel_size = 3, stride = 1),
            nn.ReLU()
        )
        
        self.fc = nn.Sequential(
            nn.Linear(in_features=64*12*12, out_features=128),
            nn.ReLU(),
            nn.Linear(in_features=128, out_features=10),
        )
    
    def forward(self, x):
        x = self.conv(x)
        x = F.max_pool2d(x,2)
        x = x.view(-1, 64*12*12)
        x = self.fc(x)
        x = F.log_softmax(x, dim=1)
        return x

In [11]:
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    # iterate over federated data
    for batch_idx, (data, target) in enumerate(train_loader):
        # send the model to the remote location 
        model = model.send(data.location)
        
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)

        # this loss is a ptr to the tensor loss at the remote location
        loss = F.nll_loss(output, target.view(-1).long())

        # call backward() on the loss ptr,that will send the command to call
        # backward on the actual loss tensor present on the remote machine
        loss.backward()
        optimizer.step()

        # get back the updated model
        model.get()

        if batch_idx % args['log_interval'] == 0:

            # a thing to note is the variable loss was also created at remote worker, so we need to explicitly get it back
            loss = loss.get()

            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, 
                    batch_idx * args['batch_size'], # no of images done
                    len(train_loader) * args['batch_size'], # total images left
                    100. * batch_idx / len(train_loader), 
                    loss.item()
                )
            )

In [12]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)

            # add losses together
            test_loss += F.nll_loss(output, target.view(-1).long(), reduction='sum').item() 
            pred = output.argmax(dim=1, keepdim=True)  
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [13]:
model = Net()
optimizer = optim.SGD(model.parameters(), lr=args['lr'])

for epoch in range(1, args['epochs'] + 1):
        train(args, model, device, federated_train_loader, optimizer, epoch)
        test(model, device, test_loader)

Train Epoch: 1 [0/60032 (0%)]	Loss: 2.300661
Train Epoch: 1 [640/60032 (1%)]	Loss: 2.300676
Train Epoch: 1 [1280/60032 (2%)]	Loss: 2.269510
Train Epoch: 1 [1920/60032 (3%)]	Loss: 2.266171
Train Epoch: 1 [2560/60032 (4%)]	Loss: 2.258022
Train Epoch: 1 [3200/60032 (5%)]	Loss: 2.253899
Train Epoch: 1 [3840/60032 (6%)]	Loss: 2.246850
Train Epoch: 1 [4480/60032 (7%)]	Loss: 2.209277
Train Epoch: 1 [5120/60032 (9%)]	Loss: 2.201339
Train Epoch: 1 [5760/60032 (10%)]	Loss: 2.171238
Train Epoch: 1 [6400/60032 (11%)]	Loss: 2.132794
Train Epoch: 1 [7040/60032 (12%)]	Loss: 2.083695
Train Epoch: 1 [7680/60032 (13%)]	Loss: 1.975187
Train Epoch: 1 [8320/60032 (14%)]	Loss: 1.946896
Train Epoch: 1 [8960/60032 (15%)]	Loss: 1.850954
Train Epoch: 1 [9600/60032 (16%)]	Loss: 1.741868
Train Epoch: 1 [10240/60032 (17%)]	Loss: 1.501534
Train Epoch: 1 [10880/60032 (18%)]	Loss: 1.401706
Train Epoch: 1 [11520/60032 (19%)]	Loss: 1.186644
Train Epoch: 1 [12160/60032 (20%)]	Loss: 0.941222
Train Epoch: 1 [12800/60032 (

Train Epoch: 2 [44800/60032 (75%)]	Loss: 0.203324
Train Epoch: 2 [45440/60032 (76%)]	Loss: 0.116181
Train Epoch: 2 [46080/60032 (77%)]	Loss: 0.290682
Train Epoch: 2 [46720/60032 (78%)]	Loss: 0.263639
Train Epoch: 2 [47360/60032 (79%)]	Loss: 0.070138
Train Epoch: 2 [48000/60032 (80%)]	Loss: 0.238364
Train Epoch: 2 [48640/60032 (81%)]	Loss: 0.224333
Train Epoch: 2 [49280/60032 (82%)]	Loss: 0.122876
Train Epoch: 2 [49920/60032 (83%)]	Loss: 0.117534
Train Epoch: 2 [50560/60032 (84%)]	Loss: 0.146825
Train Epoch: 2 [51200/60032 (85%)]	Loss: 0.241283
Train Epoch: 2 [51840/60032 (86%)]	Loss: 0.075421
Train Epoch: 2 [52480/60032 (87%)]	Loss: 0.108849
Train Epoch: 2 [53120/60032 (88%)]	Loss: 0.206410
Train Epoch: 2 [53760/60032 (90%)]	Loss: 0.244172
Train Epoch: 2 [54400/60032 (91%)]	Loss: 0.075590
Train Epoch: 2 [55040/60032 (92%)]	Loss: 0.394088
Train Epoch: 2 [55680/60032 (93%)]	Loss: 0.105463
Train Epoch: 2 [56320/60032 (94%)]	Loss: 0.237759
Train Epoch: 2 [56960/60032 (95%)]	Loss: 0.202400


Train Epoch: 4 [28800/60032 (48%)]	Loss: 0.128348
Train Epoch: 4 [29440/60032 (49%)]	Loss: 0.118972
Train Epoch: 4 [30080/60032 (50%)]	Loss: 0.085461
Train Epoch: 4 [30720/60032 (51%)]	Loss: 0.114287
Train Epoch: 4 [31360/60032 (52%)]	Loss: 0.141748
Train Epoch: 4 [32000/60032 (53%)]	Loss: 0.098521
Train Epoch: 4 [32640/60032 (54%)]	Loss: 0.270475
Train Epoch: 4 [33280/60032 (55%)]	Loss: 0.107124
Train Epoch: 4 [33920/60032 (57%)]	Loss: 0.100478
Train Epoch: 4 [34560/60032 (58%)]	Loss: 0.121096
Train Epoch: 4 [35200/60032 (59%)]	Loss: 0.042296
Train Epoch: 4 [35840/60032 (60%)]	Loss: 0.263524
Train Epoch: 4 [36480/60032 (61%)]	Loss: 0.274537
Train Epoch: 4 [37120/60032 (62%)]	Loss: 0.152648
Train Epoch: 4 [37760/60032 (63%)]	Loss: 0.049671
Train Epoch: 4 [38400/60032 (64%)]	Loss: 0.089717
Train Epoch: 4 [39040/60032 (65%)]	Loss: 0.287282
Train Epoch: 4 [39680/60032 (66%)]	Loss: 0.229501
Train Epoch: 4 [40320/60032 (67%)]	Loss: 0.024575
Train Epoch: 4 [40960/60032 (68%)]	Loss: 0.101794


Train Epoch: 6 [12800/60032 (21%)]	Loss: 0.068528
Train Epoch: 6 [13440/60032 (22%)]	Loss: 0.153918
Train Epoch: 6 [14080/60032 (23%)]	Loss: 0.226597
Train Epoch: 6 [14720/60032 (25%)]	Loss: 0.082808
Train Epoch: 6 [15360/60032 (26%)]	Loss: 0.096761
Train Epoch: 6 [16000/60032 (27%)]	Loss: 0.138324
Train Epoch: 6 [16640/60032 (28%)]	Loss: 0.071043
Train Epoch: 6 [17280/60032 (29%)]	Loss: 0.066943
Train Epoch: 6 [17920/60032 (30%)]	Loss: 0.195498
Train Epoch: 6 [18560/60032 (31%)]	Loss: 0.040779
Train Epoch: 6 [19200/60032 (32%)]	Loss: 0.099601
Train Epoch: 6 [19840/60032 (33%)]	Loss: 0.187817
Train Epoch: 6 [20480/60032 (34%)]	Loss: 0.161698
Train Epoch: 6 [21120/60032 (35%)]	Loss: 0.106130
Train Epoch: 6 [21760/60032 (36%)]	Loss: 0.018616
Train Epoch: 6 [22400/60032 (37%)]	Loss: 0.044066
Train Epoch: 6 [23040/60032 (38%)]	Loss: 0.026581
Train Epoch: 6 [23680/60032 (39%)]	Loss: 0.067570
Train Epoch: 6 [24320/60032 (41%)]	Loss: 0.027819
Train Epoch: 6 [24960/60032 (42%)]	Loss: 0.082409


Train Epoch: 7 [57600/60032 (96%)]	Loss: 0.124234
Train Epoch: 7 [58240/60032 (97%)]	Loss: 0.040691
Train Epoch: 7 [58880/60032 (98%)]	Loss: 0.235044
Train Epoch: 7 [59520/60032 (99%)]	Loss: 0.083562

Test set: Average loss: 0.0953, Accuracy: 9699/10000 (97%)

Train Epoch: 8 [0/60032 (0%)]	Loss: 0.171805
Train Epoch: 8 [640/60032 (1%)]	Loss: 0.208774
Train Epoch: 8 [1280/60032 (2%)]	Loss: 0.054451
Train Epoch: 8 [1920/60032 (3%)]	Loss: 0.140121
Train Epoch: 8 [2560/60032 (4%)]	Loss: 0.065116
Train Epoch: 8 [3200/60032 (5%)]	Loss: 0.033576
Train Epoch: 8 [3840/60032 (6%)]	Loss: 0.026378
Train Epoch: 8 [4480/60032 (7%)]	Loss: 0.054831
Train Epoch: 8 [5120/60032 (9%)]	Loss: 0.020852
Train Epoch: 8 [5760/60032 (10%)]	Loss: 0.104904
Train Epoch: 8 [6400/60032 (11%)]	Loss: 0.068007
Train Epoch: 8 [7040/60032 (12%)]	Loss: 0.057289
Train Epoch: 8 [7680/60032 (13%)]	Loss: 0.010269
Train Epoch: 8 [8320/60032 (14%)]	Loss: 0.025279
Train Epoch: 8 [8960/60032 (15%)]	Loss: 0.175086
Train Epoch: 8 [9

Train Epoch: 9 [41600/60032 (69%)]	Loss: 0.259114
Train Epoch: 9 [42240/60032 (70%)]	Loss: 0.045182
Train Epoch: 9 [42880/60032 (71%)]	Loss: 0.187853
Train Epoch: 9 [43520/60032 (72%)]	Loss: 0.066809
Train Epoch: 9 [44160/60032 (74%)]	Loss: 0.481488
Train Epoch: 9 [44800/60032 (75%)]	Loss: 0.039690
Train Epoch: 9 [45440/60032 (76%)]	Loss: 0.168031
Train Epoch: 9 [46080/60032 (77%)]	Loss: 0.091663
Train Epoch: 9 [46720/60032 (78%)]	Loss: 0.088382
Train Epoch: 9 [47360/60032 (79%)]	Loss: 0.018828
Train Epoch: 9 [48000/60032 (80%)]	Loss: 0.040167
Train Epoch: 9 [48640/60032 (81%)]	Loss: 0.075106
Train Epoch: 9 [49280/60032 (82%)]	Loss: 0.020438
Train Epoch: 9 [49920/60032 (83%)]	Loss: 0.103481
Train Epoch: 9 [50560/60032 (84%)]	Loss: 0.070574
Train Epoch: 9 [51200/60032 (85%)]	Loss: 0.277811
Train Epoch: 9 [51840/60032 (86%)]	Loss: 0.096832
Train Epoch: 9 [52480/60032 (87%)]	Loss: 0.117702
Train Epoch: 9 [53120/60032 (88%)]	Loss: 0.004692
Train Epoch: 9 [53760/60032 (90%)]	Loss: 0.024109


In [19]:
torch.save(model.state_dict(), 'MNISTCNN.pth')